In [1]:
import pandas as pd
import numpy as np

from data_processing import *
from utils import feature_desc

from sklearn.preprocessing import StandardScaler

#### Load data

In [2]:
df_og = load(csv_path='../data/merged.csv')

#### Aggregate time into clusters of size k

In [3]:
k = 15
df_og = aggregate(df_og, k)

/home/kacper/studia/cuvalley/data_processing/data_processing.py:124: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


### Proste featury wybieram albo tworze agregując


In [4]:
df = pd.DataFrame(index=df_og.index)

In [5]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = df_og.columns[df_og.columns.str.contains('tir')].values
df['TIR'] = df_og[TIR].mean(axis='columns')

In [6]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = df_og.columns[df_og.columns.str.contains('001tix')].values
df['TIX1'] = df_og[TIX1].mean(axis='columns')

In [7]:
# prob_s i prob_corg
PR = ['prob_s', 'prob_corg']
df['prob_s'] = df_og['prob_s']
df['prob_corg'] = df

In [8]:
# reg nadawy koncentratu
FCX = df_og.columns[df_og.columns.str.contains('fcx')].values
df['FCX'] = df_og[FCX].mean(axis='columns')

In [9]:
# sumaryczna moc cieplna
NIR = df_og.columns[df_og.columns.str.contains('nir')].values
df['NIR'] = df_og[NIR]

In [10]:
# WENT ODCZ ZAD OBROTÓW
UXM = df_og.columns[df_og.columns.str.contains('uxm')].values
df['UXM'] = df_og[UXM].mean(axis='columns')

In [11]:
# dodanie temperatury żużla
df["TEMP_ZUZ"] = df_og["temp_zuz"]

In [12]:
df1, df2, df3 = split(df, proportions=(0.7, 0.15, 0.15))

## Data is now ready to do somthing with it
## Eventually you can scale the data as follows

In [13]:
df1.head(3)

,TIR,TIX1,Prob,FCX,NIR,UXM,TEMP_ZUZ
czas,,,,,,,
2020-09-30 22:00:00+00:00,29.090193,418.987794,9.235,36.152944,14.384833,92.173463,1297.0
2020-09-30 22:15:00+00:00,29.054909,418.976831,9.235,36.037006,14.551195,92.173879,NaN
2020-09-30 22:30:00+00:00,29.013282,418.956943,9.235,36.102824,14.731094,92.173529,NaN


In [14]:
df1, df2, df3 = scale(df1, df2, df3)

/home/kacper/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [15]:
df1.head(3)

,TIR,TIX1,Prob,FCX,NIR,UXM,TEMP_ZUZ
czas,,,,,,,
2020-09-30 22:00:00+00:00,0.517242,0.462989,-1.690271,0.142896,-1.242959,0.087291,-1.035249
2020-09-30 22:15:00+00:00,0.463288,0.461079,-1.690271,0.130519,-1.200510,0.087324,NaN
2020-09-30 22:30:00+00:00,0.399633,0.457614,-1.690271,0.137545,-1.154607,0.087296,NaN
